## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-26-22-50-36 +0000,startribune,"Live: From Intoxicats to Ozomatli, a full day ...",https://www.startribune.com/live-music-enterta...
1,2025-08-26-22-49-36 +0000,nypost,Ex-DE Shaw trader asks NY court to ax ‘release...,https://nypost.com/2025/08/26/business/ex-de-s...
2,2025-08-26-22-47-57 +0000,startribune,"Who’s in, who’s out for the Vikings on roster ...",https://www.startribune.com/minnesota-vikings-...
3,2025-08-26-22-47-36 +0000,nyt,"U.S. Envoy Talks Peace in Lebanon, but Stirs A...",https://www.nytimes.com/2025/08/26/world/middl...
4,2025-08-26-22-46-13 +0000,nyt,SpaceX Starship Test Launch: How and When to W...,https://www.nytimes.com/2025/08/26/science/spa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
43,trump,67
58,cook,22
57,lisa,20
161,new,17
377,gaza,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
26,2025-08-26-21-52-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...,191
254,2025-08-26-03-44-00 +0000,wsj,"President Trump said he is removing Lisa Cook,...",https://www.wsj.com/politics/policy/trump-says...,191
6,2025-08-26-22-42-00 +0000,wsj,President Trump’s attempt to fire Federal Rese...,https://www.wsj.com/economy/central-banking/un...,173
9,2025-08-26-22-33-58 +0000,nypost,Fed governor Lisa Cook hires Hunter Biden lawy...,https://nypost.com/2025/08/26/us-news/fed-gove...,171
151,2025-08-26-16-12-46 +0000,wapo,Federal Reserve governor Lisa Cook says she’ll...,https://www.washingtonpost.com/business/2025/0...,164


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
26,191,2025-08-26-21-52-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...
185,62,2025-08-26-13-35-35 +0000,nypost,Netanyahu calls deadly double strike on Gaza h...,https://nypost.com/2025/08/26/world-news/netan...
196,47,2025-08-26-11-52-00 +0000,wsj,Illinois Democratic Gov. JB Pritzker promised ...,https://www.wsj.com/politics/policy/chicago-vo...
203,47,2025-08-26-10-39-53 +0000,cbc,Australia accuses Iran of being behind two ant...,https://www.cbc.ca/news/world/australia-iran-a...
27,46,2025-08-26-21-51-00 +0000,wsj,Canada’s manufacturers and wholesalers look to...,https://www.wsj.com/articles/canada-factory-wh...
79,42,2025-08-26-19-53-01 +0000,nypost,Judge rips Andrew Cuomo for using $20M in taxp...,https://nypost.com/2025/08/26/us-news/judge-ri...
62,39,2025-08-26-20-44-00 +0000,nypost,NY podiatrist hired hit man to kill his wife —...,https://nypost.com/2025/08/26/us-news/ny-podia...
33,38,2025-08-26-21-33-46 +0000,nypost,Parents of tragic little Emmanuel Haro charged...,https://nypost.com/2025/08/26/us-news/parents-...
216,36,2025-08-26-10-00-00 +0000,latimes,Hiltzik: Why are all these leading Democrats s...,https://www.latimes.com/business/story/2025-08...
186,31,2025-08-26-13-33-45 +0000,nypost,Syrian migrant arrested for brutal attack on A...,https://nypost.com/2025/08/26/world-news/syria...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
